## Pull protein sequences from NCBI
Prior to this, DAVID (DAVID Functional Annotation Bioinformatics Microarray Analysis, https://david.ncifcrf.gov/conversion.jsp ) was used to convert REF_SEQ accession numbers to Protein_GI_Accession numbers. **GI_number_generator_AK.ipynb**

After this, percent match between the input protein sequence/Entrez ID will be compared to each of the generated PROTEIN_seq.

In [2]:
# import libraries and define associated email for queries
from Bio import Entrez
import time
import pandas as pd

Entrez.email ="karanjia@uw.edu" # maybe switch this to a user input

data = pd.read_csv('GI_Numbers_combo.csv')
data.head()

,Unnamed: 0,GENENAME,ORG,ENTREZID,REFSEQ,LOG2FC,True_EID
0,0,ABC_transporter,PA,879411,NP_248876,-0.038469,15595384
1,1,ABC_transporter,PA,883108,NP_248894,-0.207718,15595400
2,2,ABC_transporter,PA,878380,NP_249014,-0.187309,15595520
3,3,ABC_transporter,PA,880771,NP_249293,0.085173,15595799
4,4,ABC_transporter,PA,879023,NP_249295,0.038834,15595801


In [12]:
type(data['True_EID'][0])

numpy.int64

In [5]:
# define gis and use Biopython.Entrez to pull protein sequences from NCBI

gis = [728886557] # gi = gi number, input is PROTEIN_GI
                         # eventually replace with column from correct df 
request = Entrez.epost("protein",id=",".join(map(str,gis)))
result = Entrez.read(request)
webEnv = result["WebEnv"]
queryKey = result["QueryKey"]
handle = Entrez.efetch(db="protein",retmode="xml", webenv=webEnv, query_key=queryKey)

d = [] # empty dataframe 

for r in Entrez.parse(handle):
    try:
        gi=int([x for x in r['GBSeq_other-seqids'] if "gi" in x][0].split("|")[1])
    except ValueError:
        gi=None
    print (">GI ",gi," " + r["GBSeq_primary-accession"] + " " + r["GBSeq_definition"] + "\n" + r["GBSeq_sequence"])
    #print (">GI ",gi," " +  r["GBSeq_sequence"])
    #d.append((gi, r['GBSeq_sequence']))
    
pd.DataFrame(d, columns=('GI', 'PROTEIN_SEQ'))

>GI  728886557  AIY94952 flagellar hook-basal body protein FlhP [Bacillus subtilis subsp. subtilis str. 168]
mlrsmltasttlnqlqqqidtissnlsnsnttgykakdtnfselvrqqfdqvdekneevakarktppglrlgvgammssrlvsdqgsiqktdrdldiaftspyqylqvnvngnrqytrdgalyvtpsaananqlqlvtgngypvldengntvnidssmknitinkngtltasdgnavqrfnlgvvqvnnpqelksegnnlfsidnaaafeelnganrqnigmqqgslemsnvdiseqmtdlitsqrsyqlnsrtitmgdqmlglinsvr


,GI,PROTEIN_SEQ


In [28]:
id_list = data["True_EID"]

d = []

#for i in range(len(id_list)):
request = Entrez.epost("protein",id=",".join(map(str,id_list)))
result = Entrez.read(request)
webEnv = result["WebEnv"]
queryKey = result["QueryKey"]
handle = Entrez.efetch(db="protein",retmode="xml", webenv=webEnv, query_key=queryKey)

for r in Entrez.parse(handle):
    try:
        gi=int([x for x in r['GBSeq_other-seqids'] if "gi" in x][0].split("|")[1])
    except ValueError:
        gi=None
    d.append((r['GBSeq_sequence']))
    
prot_seq = pd.DataFrame(d, columns=['PROTEIN_SEQ'])
prot_seq.to_csv('prot_seq.csv')

In [47]:
prot_seq

,PROTEIN_SEQ
0,mkaltssllglfaapvlagllgayvplasaappkeiriavpdvsag...
1,mhqriasiglgltlalggsaqaagqlnvvswsgyfspqllekfeke...
2,mtyrtpltllfaaglalggqaraegtlhfanwsdyyppellkkfek...
3,mlpamrtgllcallgvtapawaeyvtvisfggankeaqetafykpf...
4,mskslkaaslkfatlaaglacaaqamavdltvvsfgganksaqika...
...,...
431,mtqpskttkltkdevdrlisdyqtkqdeqaqetlvrvytnlvdmla...
432,mdmklqqvqvlkpqltqelrqaitllgyhsaelaeyidelslenpl...
433,mdvevkkngknaqlkdhevkelikqsqngdqqardllieknmrlvw...
434,msrnkveicgvdtsklpvlkneemrklfrqlqdegddsareklvng...


In [50]:
data['SEQUENCE'] = pd.DataFrame(d)

data

,Unnamed: 0,GENENAME,ORG,ENTREZID,REFSEQ,LOG2FC,True_EID,SEQUENCE
0,0,ABC_transporter,PA,879411,NP_248876,-0.038469,15595384,mkaltssllglfaapvlagllgayvplasaappkeiriavpdvsag...
1,1,ABC_transporter,PA,883108,NP_248894,-0.207718,15595400,mhqriasiglgltlalggsaqaagqlnvvswsgyfspqllekfeke...
2,2,ABC_transporter,PA,878380,NP_249014,-0.187309,15595520,mtyrtpltllfaaglalggqaraegtlhfanwsdyyppellkkfek...
3,3,ABC_transporter,PA,880771,NP_249293,0.085173,15595799,mlpamrtgllcallgvtapawaeyvtvisfggankeaqetafykpf...
4,4,ABC_transporter,PA,879023,NP_249295,0.038834,15595801,mskslkaaslkfatlaaglacaaqamavdltvvsfgganksaqika...
...,...,...,...,...,...,...,...,...
431,431,RNA_polymerase,BS,939937,NP_388354,-0.055848,728883360,mtqpskttkltkdevdrlisdyqtkqdeqaqetlvrvytnlvdmla...
432,432,RNA_polymerase,BS,936362,NP_391300,-0.140629,728886343,mdmklqqvqvlkpqltqelrqaitllgyhsaelaeyidelslenpl...
433,433,RNA_polymerase,BS,938729,NP_390226,-0.528350,728885268,mdvevkkngknaqlkdhevkelikqsqngdqqardllieknmrlvw...
434,434,RNA_polymerase,BS,939953,NP_389416,0.303125,728884442,msrnkveicgvdtsklpvlkneemrklfrqlqdegddsareklvng...


In [51]:
data.to_csv('compiled_prot_seq.csv')

### Cells below this markdown are rough drafts of the previous set of cells, keep for documentation of different options: 

In [21]:
Entrez.email = "karanjia@uw.edu"


def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene", id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        # FIXME: How generate NAs instead of causing an error with invalid IDs?
        print ("An error occurred while retrieving the annotations.")
        print ("The error returned was %s") % e
        sys.exit(-1)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key=queryKey)
    annotations = Entrez.read(data)

    print ("Retrieved %d annotations for %d genes") % (len(annotations), len(id_list))

    return annotations

In [23]:
retrieve_annotation([879411, 883108])

TypeError: sequence item 0: expected str instance, int found

In [31]:
# don't use this
from Bio import Entrez

def entry_ENTREZ(entrezID): 
    handle = Entrez.efetch(db='nucleotide', id=entrezID, rettype = 'fasta', retmode= 'text')
    f = open('%s.fasta' % singleID, 'w')
    f.write(handle.read())
    handle.close()
    f.close()

#get an id list: this makes a big search and gets a list of id 
handle = Entrez.esearch(db='genome', term="Pseudomonas aeruginosa[Orgn]", idtype="acc")
record = Entrez.read(handle)
handle.close()
print(record["IdList"])

['187']
